In [ ]:
from IPython.display import Image

# Introduction

In [ ]:
Image("../input/kernelassets/pexels-rosemary-ketchum-1464230.jpg")

This notebook covers a moderately detained analysis of US Police Shootings on the Dataset provided. Hope you find it useful and learn how to make interactive notebooks just like this one!

# Installation and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import warnings
from tqdm.notebook import tqdm

from plotly import express as px
from plotly import graph_objs as go
from plotly import figure_factory as ff

from colorama import Fore, Style

from geopy.geocoders import Nominatim

warnings.simplefilter("ignore")
plt.style.use("fivethirtyeight")
geolocator = Nominatim(user_agent="police-shooting-viz")

In [ ]:
def cout(string: str, color=Fore.RED):
    """
    Utility function to string in colors
    """
    print(color+string+Style.RESET_ALL)

In [ ]:
def statistics(dataframe, column):
    cout(f"The Average value in {column} is: {dataframe[column].mean():.2f}", Fore.RED)
    cout(f"The Standard Deviation of {column} is: {dataframe[column].std():.2f}", Fore.LIGHTCYAN_EX)
    cout(f"The Maximum value in {column} is: {dataframe[column].max()}", Fore.BLUE)
    cout(f"The Minimum value in {column} is: {dataframe[column].min()}", Fore.YELLOW)
    cout(f"The 25th Quantile of {column} is: {dataframe[column].quantile(0.25)}", Fore.GREEN)
    cout(f"The 50th Quantile of {column} is: {dataframe[column].quantile(0.50)}", Fore.CYAN)
    cout(f"The 75th Quantile of {column} is: {dataframe[column].quantile(0.75)}", Fore.MAGENTA)

In [ ]:
# Let's read the data
data = pd.read_csv("../input/us-police-shootings/shootings.csv")
data.head()

# EDA
Let's start doing EDA through each column

## Date
I will not plot whole date but rather only the month and the year of the shootings.
For this, we will draw Year and Month Barplots seperately.

In [ ]:
# Get Date and Month from the Data and store it in a seperate column
all_year = []
all_months = []

months = {
    '01':'January',
    '02':'February',
    '03':'March',
    '04':'April',
    '05':'May',
    '06':'June',
    '07':'July',
    '08':'August',
    '09':'September',
    '10':'October',
    '11':'November',
    '12':'December'
}

def get_date(datetime):
    date = str(datetime)
    year = datetime[:4]
    month = datetime[5:7]
    return year, months[month]

for date in data['date']:
    yr, mn = get_date(date)
    all_year.append(yr)
    all_months.append(mn)

data['year'] = all_year
data['month'] = all_months

In [ ]:
# Year BarPlot
targets = data['year'].value_counts().tolist()
values = list(dict(data['year'].value_counts()).keys())

fig = px.bar(
    x=values,
    y=targets,
    color=values,
    labels={'x':'Years', 'y':'Number of Incidents'},
    title="Number of Incidents over years"
)

fig.show()

In [ ]:
# Months BarPlot
targets = data['month'].value_counts().tolist()
values = list(dict(data['month'].value_counts()).keys())

fig = px.bar(
    x=values,
    y=targets,
    color=values,
    labels={'x':'Months', 'y':'Number of Incidents'},
    title="Number of Incidents over months"
)

fig.show()

## Manner of Death
Let's look at the manner of death of the individuals

In [ ]:
# Pie Chart to see the manner of death
targets = data['manner_of_death'].value_counts().tolist()
values = list(dict(data['manner_of_death'].value_counts()).keys())

fig = px.pie(
    values=targets, 
    names=values,
    title='Manner of Death in Police Shootings',
    color_discrete_sequence=['gray', 'black']
)
fig.show()

## Armament Type
Let's look if the individual that was shot had any sort of armaments on them.

In [ ]:
# Months BarPlot
targets = data['armed'].value_counts().tolist()
values = list(dict(data['armed'].value_counts()).keys())

fig = px.bar(
    x=values,
    y=targets,
    color=values,
    labels={'x':'Armament Type', 'y':'Number of Incidents'},
    title="Number of Incidents with different armaments"
)

fig.show()

## Age
Let's look at the general trends of age of individuals

In [ ]:
statistics(data, "age")

In [ ]:
# Let's plot the age column too
plt.style.use("classic")
sns.distplot(data['age'], color='blue')
plt.title(f"Age Distribution [\u03BC : {data['age'].mean():.2f} years | \u03C3 : {data['age'].std():.2f} years]")
plt.xlabel("Age")
plt.ylabel("Count")
plt.show()

As we can see, most individuals are between `25-40` years old with an average one being 36 Years old.

## Age vs Gender
Let's see a KDE Plot to see how age and gender are related, if at all

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(16, 6))
sns.kdeplot(data.loc[data['gender'] == 'M', 'age'], label = 'Male',shade=True)
sns.kdeplot(data.loc[data['gender'] == 'F', 'age'], label = 'Female',shade=True)

# Labeling of plot
plt.xlabel('Age')
plt.ylabel('Density')
plt.title('Distribution of Ages for Male and Female Individuals')
plt.show()

## Gender
Let's now take a look at Gender

In [ ]:
# Pie Chart to see gender of individuals
targets = data['gender'].value_counts().tolist()
values = list(dict(data['gender'].value_counts()).keys())

fig = px.pie(
    values=targets, 
    names=values,
    title='Gender of Individuals in Police Shootings',
    color_discrete_sequence=['blue', 'magenta']
)
fig.show()

As we can see, there is a very clear imbalance when it comes to gender of individuals who were shot in confrentation with police.

## What Armaments do Women Individuals had?
Let's see what sort of armaments women individuals had on them.

In [ ]:
targets = data[data['gender']=='F']['armed'].value_counts().tolist()
values = list(dict(data[data['gender']=='F']['armed'].value_counts()).keys())

fig = px.pie(
    values=targets, 
    names=values,
    title='Armaments held by Women during Shootings',
)
fig.show()

## Race 
Let's take a look at the race of the individual.

In [ ]:
# Pie Chart to see the manner of death
targets = data['race'].value_counts().tolist()
values = list(dict(data['race'].value_counts()).keys())

fig = px.pie(
    values=targets,
    names=values,
    title='Races of individuals in Police Shootings',
)

fig.show()

## City of Incident
Let's draw up an interactive bubble map with the city of incidents.

In [ ]:
# First, let's make a new dataframe with only cities arrange according to number of incidents uniquely
city_names = dict(data['city'].value_counts()).keys()
city_incidents = data['city'].value_counts().tolist()

city_df = pd.DataFrame()
city_df['name'] = city_names
city_df['incidents'] = city_incidents
city_df.head()

There are 12 cities that couldn't be geocoded by geopy, so we will drop them from `city_df` since there absence will not make much of a difference anyways.

In [ ]:
# Let's add the corresponding longitude and latitude to the cities.
longs, lats = [], []
err_idx = []
for idx, city in tqdm(enumerate(city_df['name'])):
    loc = geolocator.geocode(city)
    try:
        longs.append(loc.longitude)
        lats.append(loc.latitude)
    except:
        err_idx.append(idx)
        
city_df = city_df.drop(err_idx)
city_df['lat'] = lats
city_df['lon'] = longs

In [ ]:
# Draw a bubble map for city of incidents.
city_df['text'] = city_df['name'] + '<br>Incidents: ' + (city_df['incidents']).astype(str)
limits = [(0, 4), (5, 12), (12, 22), (22, 33), (33, 43)]
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
cities = []

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = city_df[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['incidents'],
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(
        title_text = 'Police Shooting Incident across US Cities<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(200, 200, 200)',
        )
    )

fig.show()

## State of Incident
Let's now turn towards individual states of United States and see how number of police shooting incidents vary from state to state.

In [ ]:
# Just like the cities, we will make a new dataframe
state_names = dict(data['state'].value_counts()).keys()
state_incidents = data['state'].value_counts().tolist()

state_df = pd.DataFrame()
state_df['state'] = state_names
state_df['incidents'] = state_incidents
state_df.head()

In [ ]:
# Choropleth map
fig = go.Figure(data=go.Choropleth(
    locations=state_df['state'], # Spatial coordinates
    z = state_df['incidents'].astype(int), # Data to be color-coded
    locationmode = 'USA-states', # set of locations
    colorscale = 'amp', # color scale
    colorbar_title = "Incidents Density", # title for the color bar
))

fig.update_layout(
    title_text = 'Police Shooting Incidents Across US States', # title for the plot
    geo_scope='usa', # limite map scope to USA
)

fig.show()

## Signs of Mental Illness
Let's see if the shot individual had any signs of mental illness

In [ ]:
# Pie Chart to see the manner of death
targets = data['signs_of_mental_illness'].value_counts().tolist()
values = list(dict(data['signs_of_mental_illness'].value_counts()).keys())

fig = px.pie(
    values=targets,
    names=values,
    title='Signs of Mental Illness in individuals',
    color_discrete_sequence=['blue', 'red']
)

fig.show()

We can see that only a quarter of shot individuals had any signs of mental illness.

## Threat Level
Let's see the threat level posed by the involved individual.

In [ ]:
# Pie Chart to see the manner of death
targets = data['threat_level'].value_counts().tolist()
values = list(dict(data['threat_level'].value_counts()).keys())

fig = px.pie(
    values=targets,
    names=values,
    title='Threat Level Posed by Individuals',
    color_discrete_sequence=['orange', 'yellow', 'gold']
)

fig.show()

## Fleeing
Let's see if how individuals involved were fleeing the crime scene

In [ ]:
# Pie Chart to see the manner of death
targets = data['flee'].value_counts().tolist()
values = list(dict(data['flee'].value_counts()).keys())

fig = px.pie(
    values=targets,
    names=values,
    title='How Individuals were Fleeing the Crime Scene',
)

fig.show()

## Body Camera
Now we look at what number of them had body camera on them.

In [ ]:
# Pie Chart to see the manner of death
targets = data['body_camera'].value_counts().tolist()
values = list(dict(data['body_camera'].value_counts()).keys())

fig = px.pie(
    values=targets,
    names=values,
    title='Possession of Body Camera',
)

fig.show()

## Arms Category
Let's also look at the category of arms possessed by individuals.

In [ ]:
# Months BarPlot
targets = data['arms_category'].value_counts().tolist()
values = list(dict(data['arms_category'].value_counts()).keys())

fig = px.bar(
    x=values,
    y=targets,
    color=values,
    labels={'x':'Armament Type', 'y':'Number of Incidents'},
    title="Number of Incidents with different armaments"
)

fig.show()

## Pandas Profiling
Finally, let's look at the Pandas Profiling Report

In [ ]:
profile = ProfileReport(data, title="Pandas Profiling Report")

In [ ]:
profile.to_notebook_iframe()

# Thank you!

Please correct me if I’ve made any mistakes in EDA, modelling or maybe explaining some concept since I am a beginner and prone to making mistakes.